## 임포트 

In [ ]:
## import selenium

from selenium import webdriver 
from selenium.webdriver import ActionChains 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time

import pygsheets

from datetime import timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd 
import math
from datetime import datetime 

from bs4 import BeautifulSoup
import numpy as np
import re

from time import sleep
import json
import matplotlib.pyplot as plt

import urllib
from urllib import request
import re

import schedule

import urllib.request
from datetime import datetime, timedelta
import pandas as pd
import json

import pandas as pd 


from powernad.API.Campaign import *
from powernad.API.RelKwdStat import *



from urllib.error import HTTPError


### 네이버 광고 API Key

In [2]:
BASE_URL = 'https://api.naver.com'
API_KEY = '***********'
SECRET_KEY = '******'
CUSTOMER_ID = '*****'

### PowerNAD 모듈 

In [3]:
rel = RelKwdStat(BASE_URL, API_KEY, SECRET_KEY, CUSTOMER_ID)

In [4]:
def search_keyword(searchword):
    

    kwdDataList = rel.get_rel_kwd_stat_list(siteId=None, biztpId=None, hintKeywords=searchword, event=None, month=None, showDetail='1')


    kwd_result = (kwdDataList[0].relKeyword, #키워드
                 kwdDataList[0].monthlyPcQcCnt, #월간 검색수 (PC)
                 kwdDataList[0].monthlyMobileQcCnt, # 월간 검색수 (Mobile)
                 kwdDataList[0].monthlyPcQcCnt+kwdDataList[0].monthlyMobileQcCnt) # 월간 total 

    return(kwd_result[3])

### 구글 API 활성화

In [5]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]
json_file_name = 'pivotal-pattern-308001-e0b7a6c7bc42.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)


In [6]:
dictionary_yester={}
dictionary_now={}

#total_keyword= pd.DataFrame(columns= ['날짜','수집날짜','쿼리수','키워드'])
#test_keyword = pd.DataFrame(columns=['수집날짜','날짜','쿼리수','키워드'])

In [7]:
gc1 = gc.open('test')
#gc1.add_worksheet(title='검수',rows=10000,cols=4)
gc3 = gc1.worksheet('검수')
#gc3.insert_row(['수집날짜', 'Error값 개수','Error 키워드'])

In [8]:
today_df=[]
yester_df=[]
spred=[]

In [9]:
error=[]

## 스케쥴러 멈춤 시 쌓아왔던 데이터 

In [10]:
keyword_query = pd.read_csv('일일키워드.csv')

In [11]:
keyword_query.drop(['Unnamed: 0'],axis=1,inplace=True)

## 무한루프_스케쥴러

In [12]:
def job() : 
    global spred
    global yester_df
    global today_df
    global error
    global keyword_query
    global new_list
    global num
    
    gc1 = gc.open('test')
    #gc1.add_worksheet(title='keyword',rows=10000,cols=4)
    
    
   # gc2 = gc1.worksheet('keyword')
   # gc2.insert_row(['날짜','수집날짜','쿼리수','키워드'])
    
    ### 키워드 
    keyword = ['키워드 리스트'
                   ]
    
    
    #빈값 제거 
    keyword = [v for v in keyword if v ]
    keyword =[v.replace(" ","") for v in keyword ]

    
    # 중복값 제거 

    keyword_list = []

    
    for i in keyword: 

        if i not in keyword_list : 
            keyword_list.append(i)
  

    
    #### 연관 검색어 리스트 
    temp = [] 

    for i in range(len(keyword_list)):
    
        sleep(0.1)
        try:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.headless = True
            chrome_options.add_argument('--headless') 
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            driver = webdriver.Chrome('./chromedriver', chrome_options=chrome_options)



            URL = "https://search.naver.com/search.naver?ie=UTF-8&sm=whl_hty&query={}".format(keyword_list[i])


            driver.implicitly_wait(3)
            driver.get(url=URL)

            driver.implicitly_wait(3)

            searches = driver.find_elements_by_css_selector(".lst_related_srch li")
          
            for keyword in searches:
                result = keyword.text
                temp.append(result)

            driver.close()


        except NameError:
            print(i)
            
    
    ### 유사 검색어 리스트 
    tempt=[]
    
    
    for tmp in range(len(keyword_list)):
        sleep(0.1)
       
        URL = 'https://www.naver.com'
        

        chrome_options = webdriver.ChromeOptions()
        chrome_options.headless = True
        chrome_options.add_argument('--headless') 
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        
        
        driver = webdriver.Chrome('./chromedriver', chrome_options=chrome_options)        
        driver.implicitly_wait(3)
        driver.get(url=URL)
        
        key = keyword_list[tmp]
        driver.find_element_by_name('query').send_keys(key)
        driver.implicitly_wait(3)

        class_names = driver.find_elements_by_css_selector('#autoFrame > div > div > div.atcmp_fixer._atcmp_layer > div.atcmp_container._words > ul>li')


        for i in class_names:
            if ((i.text != key) and(i.text is not None)) : 
                class_name = i.text

                nn= class_name.replace('\n추가',"")
                tempt.append(nn)
        sleep(0.1) 
        driver.close()

        
        

    # new 키워드 리스트 파악 

    new_keyword = temp + tempt

    #기존 검색 키워드 제거
    final= [] 

    for j in new_keyword : 
        if j not in keyword_list : 
            final.append(j)


    # 기존 단어는 뒤로 붙이기

    final = keyword_list + final 


    

    ### 네이버 데이터랩 검색어 트랜드 API


    today = str(datetime.now().date())

    
    
    # api token
    client_id = "*****" 
    client_secret = "******"



    # 날짜 

    toda = datetime.now()
    time_month = toda - relativedelta(months=1)
    time_month = time_month.strftime('%Y-%m-%d')
    time_month= str(time_month)


    yesterday = toda - relativedelta(days=1)
    yesterday = yesterday.strftime('%Y-%m-%d')
    yesterday = str(yesterday)



    today = str(datetime.now().date())


    #빈값 제거 
    final = [v for v in final if v ]
    final =[v.replace(" ","") for v in final ]



    # 중복값 제거 

    new_list = []

    for i in final: 

        if i not in new_list : 
            new_list.append(i)






    ### 리스트 분할기 도


    num =math.floor(len(new_list) /3)


    final_1=new_list[:num]
    final_2=new_list[num:num*2]
    final_3 = new_list[num*2:]

    error=[]

    
    
    ## 트랜드 지수 및 구글 스프레드 시트 

    for i in final_1:  
        sleep(1)
        try:
            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]


            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))

        except (TypeError, IndexError,KeyError,ValueError):
            print(" 타입 or 인덱스 에러,Value 에러:",i)
            error.append(i)



        except HTTPError:
            print('http 에러:', i )
            error.append(i)

            
            #여기부터 새로코딩함 

        dt_index = pd.date_range(start=time_month, end= yesterday)
        dt_list = dt_index.strftime("%Y-%m-%d").tolist()

        date = pd.DataFrame(data=dt_list, columns=['날짜'])

        spred = pd.merge(date,aa,left_on='날짜',right_on='date',how='outer')
        spred.drop(['date'],inplace=True, axis=1)

        spred.replace(np.nan,0,inplace=True)


        a=[] 

        for j in range(len(spred)) : 
            a.append(today)


        spred['수집날짜'] = a

        spred = spred[['날짜','수집날짜',i]]

        if i in dictionary_yester:

            pass

        else : 
            dictionary_yester[i]=[]



        dictionary_now[i]=spred


        # 빈 딕셔너리는 반복문 밖으로 빼도 인식되는지 파악해보기 

            
            
        try : 
            if dictionary_yester[i]==[]:
                dictionary_yester[i] = dictionary_now[i]
               # for k in range(3):
                #    sleep(1)
                 #   gc2.append_row(dictionary_now[i].iloc[-1-k,:].values.tolist()+[i])

                        
            else : 
                pass 



        except ValueError:
            
      
                yester_df= dictionary_yester[i]
                today_df = dictionary_now[i]


                yester_df.set_index('날짜',inplace=True)
             
                today_df.set_index('날짜',inplace=True)

                yester_df.update(today_df)


                tmpt = today_df.iloc[-1,:]

                yester_df = yester_df.append(tmpt)
                
                yester_df.reset_index(inplace=True)
                today_df.reset_index(inplace=True)
                
                #dictionary_yester[i] = yester_df

            ## 구글 API로 데이터 올리기
                # 2주간의 데이터만 구글 스프레드 시트에 올리겠다.

              #  for k in range(3):
               #     sleep(1)
                #    gc2.append_row(yester_df.iloc[-1-k,:].values.tolist()+[i])




                dictionary_yester[i] = yester_df


        except (APIError, NameError):
            print('API 에러',i)
            error.append(i)

       
            
    sleep(3)
    



    for i in final_2:  
        sleep(1)
        try:
            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]


            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))

            

        except (TypeError,IndexError,KeyError,ValueError):
            print(" 타입 or 인덱스 에러, Value 에러:",i)
            error.append(i)



        except HTTPError:
            print('http 에러:', i )
            error.append(i)
      

        #여기부터 새로코딩함 

        dt_index = pd.date_range(start=time_month, end= yesterday)
        dt_list = dt_index.strftime("%Y-%m-%d").tolist()

        date = pd.DataFrame(data=dt_list, columns=['날짜'])

        spred = pd.merge(date,aa,left_on='날짜',right_on='date',how='outer')
        spred.drop(['date'],inplace=True, axis=1)

        spred.replace(np.nan,0,inplace=True)


        a=[] 

        for j in range(len(spred)) : 
            a.append(today)

        spred['수집날짜'] = a

        spred = spred[['날짜','수집날짜',i]]

        if i in dictionary_yester:

            pass

        else : 
            dictionary_yester[i]=[]



        dictionary_now[i]=spred

        # 빈 딕셔너리는 반복문 밖으로 빼도 인식되는지 파악해보기 

        try : 
            if dictionary_yester[i]==[]:

                dictionary_yester[i] = dictionary_now[i]
                

              #  for k in range(3):
               #     sleep(1)
                #    gc2.append_row(dictionary_now[i].iloc[-1-k,:].values.tolist()+[i])
            else : 
                pass 



        except ValueError:
            

                
                yester_df= dictionary_yester[i]
                today_df = dictionary_now[i]


                yester_df.set_index('날짜',inplace=True)
                today_df.set_index('날짜',inplace=True)

                yester_df.update(today_df)


                tmpt = today_df.iloc[-1,:]

                yester_df = yester_df.append(tmpt)
                today_df.reset_index(inplace=True)
                yester_df.reset_index(inplace=True)
                
                #dictionary_yester[i] = yester_df

            ## 구글 API로 데이터 올리기
                # 2주간의 데이터만 구글 스프레드 시트에 올리겠다.

             #   for k in range(3):
              #      sleep(1)
               #     gc2.append_row(yester_df.iloc[-1-k,:].values.tolist()+[i])




                dictionary_yester[i] = yester_df

        except (APIError,NameError):
            print('API 에러',i)
            error.append(i)


  
            
    sleep(3)

    
    

    for i in final_3:  
        sleep(1)
        try:
            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]


            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))



        except (TypeError, IndexError,KeyError, ValueError):
            print(" 타입 or 인덱스 에러 or Value에러:",i)
            error.append(i)



        except HTTPError:
            print('http 에러:', i )
            error.append(i)
            
 
        #여기부터 새로코딩함 

        dt_index = pd.date_range(start=time_month, end= yesterday)
        dt_list = dt_index.strftime("%Y-%m-%d").tolist()

        date = pd.DataFrame(data=dt_list, columns=['날짜'])

        spred = pd.merge(date,aa,left_on='날짜',right_on='date',how='outer')
        spred.drop(['date'],inplace=True, axis=1)

        spred.replace(np.nan,0,inplace=True)


        a=[] 

        for j in range(len(spred)) : 
            a.append(today)

        spred['수집날짜'] = a

        spred = spred[['날짜','수집날짜',i]]

        if i in dictionary_yester:

            pass

        else : 
            dictionary_yester[i]=[]



        dictionary_now[i]=spred


    # 빈 딕셔너리는 반복문 밖으로 빼도 인식되는지 파악해보기 

        try : 
            if dictionary_yester[i]==[]:

                dictionary_yester[i] = dictionary_now[i]

              #  for k in range(3):
                    
               #     sleep(1)
                #    gc2.append_row(dictionary_now[i].iloc[-1-k,:].values.tolist()+[i])
            else : 
                pass 



        except ValueError:
            

                yester_df= dictionary_yester[i]
                today_df = dictionary_now[i]


                yester_df.set_index('날짜',inplace=True)
                today_df.set_index('날짜',inplace=True)

                yester_df.update(today_df)


                tmpt = today_df.iloc[-1,:]

                yester_df = yester_df.append(tmpt)

                yester_df.reset_index(inplace=True)
                today_df.reset_index(inplace=True)
                    
                #dictionary_yester[i] = yester_df
                
            ## 구글 API로 데이터 올리기
                # 2주간의 데이터만 구글 스프레드 시트에 올리겠다.

            #    for k in range(3):
             #       sleep(1)
              #      gc2.append_row(yester_df.iloc[-1-k,:].values.tolist()+[i])




                dictionary_yester[i] = yester_df
           
   

        except (APIError,NameError):
            print('API 에러',i)
            error.append(i)

        
    
    
    
    sleep(3)
      
    
    
#total 키워드 한눈에 보기     
    total_keyword= pd.DataFrame(columns= ['날짜','수집날짜','쿼리수','키워드'])
    
    for i in dictionary_yester : 
        for  j in range(len(dictionary_yester[i])):
            row = dictionary_yester[i].iloc[j,:].values.tolist()+[i]

            
            
            total_keyword = total_keyword.append(pd.Series(row, index=total_keyword.columns), ignore_index=True)

            
    
    
    
    
    #추가됨 
    
    aaa = keyword_query['키워드'].values.tolist()
    
    tmp2=[]
    
    final_keyword= pd.DataFrame(columns= ['날짜','수집날짜','쿼리수','키워드'])
     
    #기존 데이터 프레임의 키워드 리스트 
    
    for i in aaa : 
        if i not in tmp2: 
            tmp2.append(i)
    
    for i in dictionary_yester:
        if i not in tmp2 : 
            final_keyword = final_keyword.append(total_keyword[total_keyword['키워드']==i])
        else: 
            pass
    
    
    try:
        for k in tmp2: 
            #기존 데이터 
            tmp3 = keyword_query[keyword_query['키워드']==k]

            tmp3.set_index('날짜', inplace=True)

            if k not in dictionary_yester : 

                tmp3.reset_index(inplace=True)

                final_keyword = final_keyword.append(tmp3)

            else : 
                tmp4 = total_keyword[total_keyword['키워드']==k]
                tmp4 = tmp4.drop_duplicates(['날짜'])   # 방지 : 같은 날 2번 연속 돌리면 같은 날짜가 2번쌓임
                tmp4.set_index('날짜',inplace=True)
                tmp3 = tmp3.copy()
                tmp3.update(tmp4)

                tmp3.reset_index(inplace=True)
                tmp4.reset_index(inplace=True)

                gg = tmp3.iloc[-1,:][0] #3월 29일 

                if gg not in tmp4['날짜'].values.tolist():

                    tmp3 = tmp3.append(tmp4)

                    final_keyword = final_keyword.append(tmp3)
                else:


                    idx = tmp4[tmp4['날짜']==gg].index.values.tolist()[0]

                    num = tmp4.iloc[idx+1:,:]

                    tmp3 = tmp3.append(num)

                    final_keyword = final_keyword.append(tmp3)

    except ValueError:
         print(i)
            
            
    final_keyword.to_csv('합침키워드.csv',encoding='utf-8-sig') 
            
    
    
    
    
    
    num=len(set(final_keyword[final_keyword['수집날짜']==today]['키워드']))
    
    gc3.append_row([today,len(error),",".join(error)])
    
    
    # 중간에 키워드를 수정하가나 하기위해 파이썬을 멈추면 해당 파일을 일일키워드(10번 블록)로 바꾸고 다시 돌리자
    total_keyword.to_csv('total_keyword.csv',encoding='utf-8-sig')
    
    #numList : 오늘 기준 연관 유사 검색어 리스트 (한달 총합 데이터 오류값 키워드 포함)
    #num : 누적된 연관 유사 검색어 리스트 (한달 총합 데이터 오류값 키워드 포함)
    
    
    
## 12시간 뒤 스프레드 시트 삭제 


    #12시간 멈춘 후 워크시트 삭제 
   # sleep(43200)
    
  #  gc1.del_worksheet(gc2)

In [13]:
#job()

In [ ]:
schedule.every().day.at("시간").do(job)   #ex_ 11:00
#schedule.every(5).minutes.do(job)
while True:
    schedule.run_pending()
    
    time.sleep(1)

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:73: DeprecationWarning: use options instead of chrome_options
/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:115: DeprecationWarning: use options instead of chrome_options


 타입 or 인덱스 에러,Value 에러: 유니셰프후원
 타입 or 인덱스 에러,Value 에러: 초록우산어린이재단해지
 타입 or 인덱스 에러,Value 에러: 월드비젼해지
 타입 or 인덱스 에러, Value 에러: 유니셰프링
 타입 or 인덱스 에러 or Value에러: 뷔루이비통팔찌
 타입 or 인덱스 에러 or Value에러: 유니셰프팔찌
 타입 or 인덱스 에러 or Value에러: 모자뜨기클럽
 타입 or 인덱스 에러 or Value에러: 굿네이버스편지쓰기상장
 타입 or 인덱스 에러 or Value에러: 유니세프후원하기
 타입 or 인덱스 에러 or Value에러: 불가리세이브더칠드런반지커플
